In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# AGIPD geometry

In [ ]:
from extra_redu.spi.tests.mock.agipd import AGIPDGeometry

geom = AGIPDGeometry(downsample=8)
geom.draw(True)

# Simple models

1. Gas background simplified fenomenological model

In [ ]:
from extra_redu.spi.tests.mock.models import gas_scattering

r = np.linspace(np.min(geom.r), np.max(geom.r))
g = type("MockGeom", (), dict(r=r))
plt.semilogy(r, gas_scattering(g))
plt.show()

2. Pulse intensities

In [ ]:
from extra_redu.spi.tests.mock.models import pulse_intensities

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4),
                               width_ratios=(3, 1))
inten = pulse_intensities(35100)
ax1.plot(inten[:351])
ax2.hist(inten, 300)
plt.show()

3. Intensity at random point of beam spot

In [ ]:
from extra_redu.spi.tests.mock.models import interaction_point_intensity

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4),
                               width_ratios=(3, 1))
inten = interaction_point_intensity(35100)
ax1.plot(inten[:351], '.')
ax2.hist(inten, 300)
ax2.semilogy()
plt.show()

4. Ball scattering

In [ ]:
from extra_redu.spi.tests.mock.models import ball_scattering, agipd_noise

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
det_data = ball_scattering(geom)
geom.draw(det_data, ax=ax1)
geom.draw(agipd_noise(5 * det_data), ax=ax2)
plt.show()

# Full SPI ball model

1. Simulate data

In [ ]:
from extra_redu.spi.tests.mock.models import spi_ball_scattering

num_cells = 31
num_trains = 50
num_pulses = num_trains * num_cells
adu_threshold = 0.7

model_param = dict( 
    photon_en = 6.,  #keV
    L = 1.7,  # m
    R = 35,  # nm
    flow_size = 12,  # um
    beam_size = 1.5,  # um

    pulse_energy = 1.8,  # mJ
    pulse_energy_rmsd = 0.010,  # mJ

    # gas parameters
    gas_ambient = 0.02,
    gas_shape = 4e-5,
)

adu = spi_ball_scattering(geom, num_pulses, **model_param)
mask = geom.random_mask(num_cells, num_trains)

2. Plot examples

In [ ]:
litpx = np.sum(adu > adu_threshold, axis=(1,2,3))
tid = np.tile(np.arange(num_cells), num_trains)
k = np.argmax(litpx)

fig, ax = plt.subplots(1, 1, figsize=(16, 3))
ax.plot(tid, litpx, '.')
plt.show()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12),
                                             height_ratios=(3, 1))

im = geom.assemble(adu[k])
ax1.matshow(im, norm='linear', vmin=adu_threshold)
ax3.hist(adu[k].flatten(), 100, range=(-0.5, 10.5))
ax3.semilogy()

im = geom.assemble(adu[0])
ax2.matshow(im, norm='linear', vmin=adu_threshold)
ax4.hist(adu[0].flatten(), 100, range=(-0.5, 10.5))
ax4.semilogy()
plt.show()

3. Moke data collection

In [ ]:
from extra_redu.fileutils.tests.mock.extra_data import MockDataCollection, MockSourceData
from extra_redu.fileutils import StackedPulseSource

train_ids = list(range(10001, 10001 + num_trains))
detector_id = "SPB_DET_AGIPD1M-1"
source_names = [f"{detector_id}/CORR/{modno}CH0:output"
                for modno in range(16)]

cellId = np.tile(np.arange(num_cells, dtype=np.int32) + 1, num_trains)[:, None]
pulseId = cellId.astype(np.uint64) * 4
trainId = np.repeat(np.array(train_ids, dtype=np.uint64), num_cells)[:, None]
count = np.full(num_trains, num_cells)
data = {}
for modno, source_id in enumerate(source_names):
    module_data = {
        "image.data": adu[:, modno],
        "image.mask": mask[:, modno],
        "image.cellId": cellId,
        "image.pulseId": pulseId,
        "image.trainId": trainId,
    }
    data[source_id] = MockSourceData(source_id, train_ids, count, module_data)

dc = MockDataCollection.mock(train_ids, data=data)

sources_ptrn = detector_id + r"/CORR/(?P<key>\d+)CH0:output"
src = StackedPulseSource.from_datacollection(dc, sources_ptrn, "image")

3. Compute lit-pixels

In [ ]:
from extra_redu.spi import LitPixelCounter, SpiHitfinder

litpx = LitPixelCounter(src)
litpx(src)

a = np.hstack([litpx.num_lit_px, np.full([litpx.num_events, 1], np.nan)])
plt.matshow(a.reshape(num_cells * 10, -1).T)
plt.show()

4. Find hits

In [ ]:
hitfinder = SpiHitfinder(
    modules=[3,4,8,15],
    min_scores=30,
    hitrate_window_size=5,
    num_px_per_module = geom.nss * geom.nfs
)
hitfinder.find_hits(litpx)

In [ ]:
hitfinder.plot_hitrate()
hitfinder.plot_hitscore_hist(num_bins=100)
hitfinder.plot_hitscore()